In [ ]:
#%pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
#%pip install bitsandbytes einops wandb -Uqqq

In [ ]:
import torch
import glob
import re
import pandas as pd
import numpy as np
import re
import wandb
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
from datasets import Dataset
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
# Initialize WandB
wandb.init(project='tiny-llama', entity='tomislav-krog')

In [ ]:
def concatenate_lyrics(df, artist_name):
    # Filter the DataFrame for the given artist
    artist_df = df[df['Artist'] == artist_name]

    # Concatenate all lyrics into a single string
    all_lyrics = '\n'.join(artist_df['Lyric'])

    return all_lyrics

In [ ]:
# Load the dataset
file_path = '/content/drive/MyDrive/all_albums/dataset.csv'
df = pd.read_csv(file_path)

# Prompt the user for an artist name
artist_name = input("Enter the name of the artist: ")

In [ ]:
# Get concatenated lyrics
lyrics = concatenate_lyrics(df, artist_name)

In [ ]:

print(' '.join(sorted(set(lyrics))))

In [ ]:
pattern = r'\[.*?\]'
lyrics = re.sub(pattern, '', lyrics)

In [ ]:
pattern = r'\((chorus|CHORUS|verse|VERSE|intro|INTRO)(.*?)\)'
lyrics =  re.sub(pattern, '', lyrics)

In [ ]:
def special_symbols_resolver(s):
    replacements = {
        'à': 'a', 'á': 'a', 'â': 'a', 'ã': 'a', 'ä': 'a',
        'ç': 'c',
        'ö': 'o',
        'ú': 'u', 'ü': 'u',
        'œ': 'oe',
        'Â': 'A',
        '‰': '', '™': '', '´': '', '·': '', '¦': '', '': '', '': '',
        '˜': '', '“': '', '†': '', '…': '', '′': '', '″': '', '�': '',
        'í': 'i', 'é': 'e', 'ï': 'i', 'ó': 'o', ';': ',', '‘': '\'', '’': '\'', ':': ',', 'е': 'e'
    }

    for symbol, replacement in replacements.items():
        s = s.replace(symbol, replacement)

    return s

In [ ]:
lyrics = special_symbols_resolver(lyrics)

In [ ]:
replace_with_space = ['\u2005', '\u200b', '\u205f', '\xa0', '-']
replace_letters = {}
remove_list = ['\)', '\(', '–','"','”', '"', '\[.*\]', '.*\|.*', '—', '(VERSE)', '(CHORUS ONE)']

cleaned_lyrics = lyrics

for string in remove_list:
    cleaned_lyrics = re.sub(string,'',cleaned_lyrics)
for string in replace_with_space:
    cleaned_lyrics = re.sub(string,' ',cleaned_lyrics)
print(''.join(sorted(set(cleaned_lyrics))))

In [ ]:
print(lyrics)

In [ ]:
split_point = int(len(cleaned_lyrics)*0.95)
train_data = cleaned_lyrics[:split_point]
test_data = cleaned_lyrics[split_point:]
train_data_seg = []
for i in range(0, len(train_data), 500):
        text = train_data[i:min(i+500, len(train_data))]
        train_data_seg.append(text)
train_data_seg = Dataset.from_dict({'text':train_data_seg})
print(len(train_data_seg))

In [ ]:
# Loading the model with double quantization
model_name = "PY007/TinyLlama-1.1B-step-50K-105b"
# model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
# model_name = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def generate_lyrics(query, model):
    encoding = tokenizer(query, return_tensors="pt").to(device)
    generation_config = GenerationConfig(max_new_tokens=250, pad_token_id = tokenizer.eos_token_id,repetition_penalty=1.3, eos_token_id = tokenizer.eos_token_id)
    outputs = model.generate(input_ids=encoding.input_ids, generation_config=generation_config)
    text_output = tokenizer.decode(outputs[0],skip_special_tokens=True)
    print('INPUT\n', query, '\n\nOUTPUT\n', text_output[len(query):])
generate_lyrics(test_data[500:1000], model)

In [ ]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32
lora_dropout = 0.05
lora_rank = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM")

peft_model = get_peft_model(model, peft_config)

In [ ]:
output_dir = "Tkrogg/TinyLlama_taylor_swift"
per_device_train_batch_size = 3
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_strategy="steps"
save_steps = 10
logging_steps = 10
learning_rate = 2e-3
max_grad_norm = 0.3
max_steps = 200
warmup_ratio = 0.03
lr_scheduler_type = "cosine"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    report_to='wandb'
)

In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_data_seg,
    peft_config=peft_config,
    max_seq_length=500,
    dataset_text_field='text',
    tokenizer=tokenizer,
    args=training_arguments
)
peft_model.config.use_cache = False

In [ ]:
trainer.train()

In [ ]:
generate_lyrics(test_data[500:1000], model)